In [4]:
!pip install gradio

In [12]:
!pip install gtts


In [1]:
import cv2
import numpy as np
from PIL import Image
import gradio as gr
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
from gtts import gTTS
import tempfile

def text_to_speech(text):
    tts = gTTS(text)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    temp_file_path = temp_file.name
    tts.save(temp_file_path)
    return temp_file_path



IMAGE_SIZE = 256

# Load Trained Model
MODEL_PATH = "plant25.h5"  # Ensure the model is saved here
model = tf.keras.models.load_model(MODEL_PATH)

# Load Disease Information
disease_info_path = "disease_info.csv"
disease_df = pd.read_csv(disease_info_path)
disease_dict = disease_df.set_index("disease_name").to_dict(orient="index")

# Load Class Names
CLASS_NAMES = ['Bacteria', 'Fungi', 'Healthy', 'Nematode', 'Pest', 'Phytopthora', 'Virus']  # Update with actual class names

# Function to preprocess the image
def preprocess_image(image):
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image
    
# Function to estimate disease severity percentage
def estimate_severity_percentage(confidence):
    return round(confidence * 100, 2)

# Function to categorize severity
def estimate_severity(confidence):
    if confidence > 0.75:
        return "Severe"
    elif confidence > 0.5:
        return "Moderate"
    else:
        return "Mild"
        
def predict_leaf_disease(image):
    img_array = preprocess_image(image)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    
    confidence = np.max(prediction)
    severity_percentage = estimate_severity_percentage(confidence)

    disease_name = CLASS_NAMES[predicted_class]
    severity = estimate_severity(confidence)
    
    disease_details = disease_dict.get(disease_name, {"cause": "Unknown", "prevention": "No information available"})

   
    result_text = (
        f"Disease: {disease_name}\n"
        f"Severity: {severity} ({severity_percentage}%)\n"
        f"Cause: {disease_details['cause']}\n"
        f"Prevention: {disease_details['prevention']}"
    )
    
  
    audio_path = text_to_speech(result_text)

    return result_text, disease_name, audio_path



def chatbot_response(user_input):
    user_input = user_input.strip().lower()
    for disease, details in disease_dict.items():
        if disease.lower() in user_input:
            response_text = (
                f"Disease: {disease}\n"
                f"Cause: {details['cause']}\n"
                f"Prevention: {details['prevention']}"
            )
            audio_path = text_to_speech(response_text)
            return [(user_input, response_text)], audio_path
    
    default_response = "Sorry, I couldn't find information on that disease."
    audio_path = text_to_speech(default_response)
    return [(user_input, default_response)], audio_path
    

def capture_frame():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        return None, "Error: Could not open camera."

    ret, frame = cap.read()
    cap.release()

    if not ret:
        return None, "Error: Could not capture frame."

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame_rgb, "Frame captured successfully."

def is_leaf(image):
    """Detect if the captured image contains a leaf based on color segmentation."""
    image = np.array(image)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

  
    lower_green = np.array([25, 40, 40])   
    upper_green = np.array([90, 255, 255])  

   
    mask = cv2.inRange(hsv, lower_green, upper_green)
    green_percentage = (cv2.countNonZero(mask) / mask.size) * 100

    return green_percentage > 10


def real_time_camera():
    frame, status = capture_frame()
    if frame is None:
        return None, status, None, None, status

    pil_image = Image.fromarray(frame)

   
    if not is_leaf(pil_image):
        result_text = "Invalid: No leaf detected. Please capture an image of a leaf."
        audio_path = text_to_speech(result_text)
        return pil_image, result_text, "No Leaf Detected", audio_path, status

    # If a leaf is detected, predict the disease
    result_text, disease_name, audio_path = predict_leaf_disease(pil_image)

    return pil_image, result_text, disease_name, audio_path, status


def video_stream_analysis(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None, f"Error: Could not open video file: {video_path}", None

    ret, frame = cap.read()
    if not ret:
        return None, "Error: Could not read video frame.", None

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)

    result_text, disease_name, audio_path = predict_leaf_disease(pil_image)

    cap.release()

    return pil_image, result_text, disease_name, audio_path, "Video analysis completed."



with gr.Blocks() as demo:
    gr.Markdown("# 🌿 Leaf Disease Detection App")
    gr.Markdown("Upload a leaf image, use your camera, or analyze a video stream to detect diseases.")

    # Tab 1: Upload Image
    with gr.Tab("Upload Image"):
        gr.Markdown("### Upload a leaf image to detect disease.")
        with gr.Row():
            image_input = gr.Image(type="pil", label="Upload Leaf Image")
        submit_button = gr.Button("Submit", variant="primary")
        with gr.Row():
            output_text = gr.Markdown(label="Prediction Result")
            output_label = gr.Label(label="Predicted Disease")
            audio_output = gr.Audio(label="Voice Output")  # Audio output added
        submit_button.click(
            fn=predict_leaf_disease,
            inputs=[image_input],
            outputs=[output_text, output_label, audio_output]
        )

  
    with gr.Tab("Real-Time Camera"):
        gr.Markdown("### Capture an image using your camera for real-time analysis.")
        camera_button = gr.Button("Capture Image", variant="primary")
        with gr.Row():
            camera_output = gr.Image(label="Captured Image")
            camera_text = gr.Markdown(label="Prediction Result")
            camera_label = gr.Label(label="Predicted Disease")
            camera_audio = gr.Audio(label="Voice Output")
            camera_status = gr.Textbox(label="Camera Status")
        camera_button.click(real_time_camera, outputs=[camera_output, camera_text, camera_label, camera_audio, camera_status])


   
    with gr.Tab("Video Stream Analysis"):
        gr.Markdown("### Upload a video for plant disease analysis.")
        video_input = gr.Video(label="Upload Video")
        video_button = gr.Button("Analyze Video", variant="primary")
        with gr.Row():
            video_output = gr.Image(label="Video Frame")
            video_text = gr.Markdown(label="Prediction Result")
            video_label = gr.Label(label="Predicted Disease")
            video_audio = gr.Audio(label="Voice Output")
            video_status = gr.Textbox(label="Analysis Status")
    video_button.click(video_stream_analysis, inputs=[video_input], outputs=[video_output, video_text, video_label, video_audio, video_status])
        #chatboat
    with gr.Tab("Chatbot"):
        chatbot = gr.Chatbot()
        user_input = gr.Textbox(placeholder="Ask about plant diseases...")
        send_button = gr.Button("Ask")
        audio_output = gr.Audio(label="Voice Output")  # Audio output added
        send_button.click(chatbot_response, inputs=[user_input], outputs=[chatbot, audio_output])       


if __name__ == "__main__":
    demo.launch(share=True)

C:\Users\HP\anaconda3\Lib\site-packages\gradio\components\chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/step


Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\anaconda3\Lib\site-packages\gradio\blocks.py", line 2106, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\anaconda3\Lib\site-packages\gradio\blocks.py", line 1862, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\anaconda3\Lib\site-packages\gradio\blocks.py", line 1817, in validate_outputs
    rai

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
